In [30]:
import yfinance as yf
import pandas as pd
import numpy as np
from textblob import TextBlob


In [31]:

# Define the ticker and the time period
ticker = 'AAPL'
start_date = '2020-04-01'
end_date = '2023-06-01'

# Fetch the stock data
ap_data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [5]:
ap_data.head()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2020-04-01,58.528080,60.227501,62.180000,59.782501,61.625000,176218400
2020-04-02,59.504726,61.232498,61.287498,59.224998,60.084999,165934000
2020-04-03,58.649563,60.352501,61.424999,59.742500,60.700001,129880000
2020-04-06,63.765984,65.617500,65.777496,62.345001,62.724998,201820400
2020-04-07,63.027439,64.857498,67.925003,64.750000,67.699997,202887200


In [32]:
ap_data.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']

ap_data.columns

Index(['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

In [33]:
# Create a DataFrame with dates from the AAPL data
dates = ap_data.index
#text_data = pd.DataFrame(dates, columns=['Date'])

In [34]:
text_data=pd.read_csv("../data/raw_analyst_ratings.csv",parse_dates=["date"])


print(text_data.head())
#print(text_data.info())

   Unnamed: 0                                           headline  \
0           0            Stocks That Hit 52-Week Highs On Friday   
1           1         Stocks That Hit 52-Week Highs On Wednesday   
2           2                      71 Biggest Movers From Friday   
3           3       46 Stocks Moving In Friday's Mid-Day Session   
4           4  B of A Securities Maintains Neutral on Agilent...   

                                                 url          publisher  \
0  https://www.benzinga.com/news/20/06/16190091/s...  Benzinga Insights   
1  https://www.benzinga.com/news/20/06/16170189/s...  Benzinga Insights   
2  https://www.benzinga.com/news/20/05/16103463/7...         Lisa Levin   
3  https://www.benzinga.com/news/20/05/16095921/4...         Lisa Levin   
4  https://www.benzinga.com/news/20/05/16095304/b...         Vick Meyer   

                        date stock  
0  2020-06-05 10:30:54-04:00     A  
1  2020-06-03 10:45:20-04:00     A  
2  2020-05-26 04:30:07-04:00 

In [35]:
text_data.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [36]:
 #Aligning sentiment scores with the stock closing prices on the same date
combined_data = text_data.set_index('date').join(ap_data['Close'])
combined_data.head()

,Unnamed: 0,headline,url,publisher,stock,Close
date,,,,,,
2020-06-05 10:30:54-04:00,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,A,NaN
2020-06-03 10:45:20-04:00,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,A,NaN
2020-05-26 04:30:07-04:00,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,A,NaN
2020-05-22 12:45:06-04:00,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,A,NaN
2020-05-22 11:38:59-04:00,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,A,NaN


In [37]:
# Convert dates to a consistent format
text_data['date'] = pd.to_datetime(text_data['date']).dt.date
ap_data['Date'] = pd.to_datetime(ap_data['Date']).dt.date

# Merge datasets on 'date'
merged_df = pd.merge(text_data, ap_data, on='date', how='inner')

ValueError: time data "2020-05-22 00:00:00" doesn't match format "%Y-%m-%d %H:%M:%S%z", at position 10. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [38]:
from textblob import TextBlob

# Define a function to calculate sentiment scores
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  # Polarity score between -1 to +1

# Apply sentiment analysis to each headline
text_data['sentiment_score'] = text_data['headline'].apply(get_sentiment)

# Aggregate average daily sentiment scores
daily_sentiment = text_data.groupby('date')['sentiment_score'].mean().reset_index()


In [ ]:
# Compute daily returns as percentage change
ap_data['daily_return'] = ap_data['Close'].pct_change()

# Remove NaN values created by pct_change
ap_data = ap_data.dropna()
